In [6]:
from basic_train import *


def main():
    # Set Training Parameters
    num_workers = 12
    bs = 64
    n_way = 3
    path_pretrained = '../results/basic/basic_36.pth'
    save_models = True  # Whether to save the trained models (Occurs every epoch)
    k_shot = 20  # Must have the generated split to match it

    path_splits = f'../splits/{k_shot}_shot.csv'  # Location of preprocessed splits
    path_results = f'../../results/{k_shot}shot_nc.csv'  # Full path to save the CSV results
    path_models = f'../../models/nc/{k_shot}_shot'  # Folder path to save the trained models to

    torch.cuda.set_device(0)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    criterion = nn.CrossEntropyLoss()
    
    # Load in model
    model = CosineSimilarityNet(n_way).to(device)
    pretrained_dict = torch.load(path_pretrained)
    del pretrained_dict['linear.weight']  # Remove the last layer
    del pretrained_dict['linear.bias']
    #del pretrained_dict['cos_sim.weight']
    model_dict = model.state_dict()
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

    # Load in data
    train_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                                csv_path=path_splits, mode='novel_train', resize=224)
    test_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                               csv_path=path_splits, mode='novel_validate', resize=224)
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)

    # Create Dataframe to export results to CSV
    df_results = pd.DataFrame(columns=['Validation Loss', 'Accuracy', 'Macro Accuracy',
                                       'Macro-F1 Score'] + [str(x) + ' F1' for x in range(n_way)])
    
    # Find Average Features
    label_features = [torch.FloatTensor([]).to(device) for i in range(n_way)]
    model.eval()
    fc_weight = torch.FloatTensor([]).to(device) # Initialise weight for the nearest centroid, last layer weight
    with torch.no_grad():
        for step, (data_inputs, data_labels) in enumerate(train_loader):
            # Get Features
            inputs, labels = data_inputs.to(device), data_labels.to(device)
            _, features = model(inputs, extract_features=True)
            
            # Sort features into labels
            for i in range(features.size(0)):
                label = labels[i]
                label_features[label] = torch.cat((label_features[label], features[i][None]))
        
        # Create weight for the last layer
        for j in range(n_way):
            feature_avg = torch.mean(label_features[j], 0)
            fc_weight = torch.cat((fc_weight, feature_avg[None]), 0)
            
        # Apply weight to the model
        nc_dict = model.state_dict()
        nc_dict['cos_sim.weight'] = fc_weight
        model.load_state_dict(nc_dict)
        
    # Testing
    val_loss, acc, m_acc, macro_f1, class_f1 = test(model, test_loader, criterion, device, n_way)

    if (save_models):
        torch.save(model.state_dict(), os.path.join(path_models, f'nc.pth'))  # Save the model

    # Append and report results
    df_results.loc[0] = [val_loss, acc, m_acc, macro_f1] + class_f1
    print(f'v_loss: {val_loss:.5f} val_acc: {acc:.5f} val_m_acc: {m_acc:.5f} f1: {macro_f1:.5f}')

    df_results.to_csv(path_results, index=False)  # Export results to a CSV file

if __name__ == '__main__':
    main()


v_loss: 1.09913 val_acc: 0.35444 val_m_acc: 0.35444 f1: 0.33954


In [3]:
from basic_train import *


def main():
    for stuff in range(1,37):
        # Set Training Parameters
        num_workers = 12
        bs = 64
        n_way = 3
        path_pretrained = f'../../models/basic/basic_{stuff}.pth'
        save_models = False  # Whether to save the trained models (Occurs every epoch)
        k_shot = 20  # Must have the generated split to match it

        path_splits = f'../splits/{k_shot}_shot.csv'  # Location of preprocessed splits
        #path_results = f'../../results/{k_shot}shot_cosine.csv'  # Full path to save the CSV results
        #path_models = f'../../models/cosine_similarity/{k_shot}_shot'  # Folder path to save the trained models to

        torch.cuda.set_device(0)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        criterion = nn.CrossEntropyLoss()

        # Load in model
        model = CosineSimilarityNet(n_way).to(device)
        pretrained_dict = torch.load(path_pretrained)
        del pretrained_dict['linear.weight']  # Remove the last layer
        del pretrained_dict['linear.bias']
        #del pretrained_dict['cos_sim.weight']
        model_dict = model.state_dict()
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)

        # Load in data
        train_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                                    csv_path=path_splits, mode='novel_train', resize=224)
        test_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                                   csv_path=path_splits, mode='novel_validate', resize=224)
        train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)
        test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)

        # Create Dataframe to export results to CSV
        df_results = pd.DataFrame(columns=['Epoch', 'Validation Loss', 'Accuracy', 'Macro Accuracy',
                                           'Macro-F1 Score'] + [str(x) + ' F1' for x in range(n_way)])

        # Find Average Features
        label_features = [torch.FloatTensor([]).to(device) for i in range(n_way)]
        model.eval()
        fc_weight = torch.FloatTensor([]).to(device) # Initialise weight for the nearest centroid, last layer weight
        with torch.no_grad():
            for step, (data_inputs, data_labels) in enumerate(train_loader):
                # Get Features
                inputs, labels = data_inputs.to(device), data_labels.to(device)
                _, features = model(inputs, extract_features=True)

                # Sort features into labels
                for i in range(features.size(0)):
                    label = labels[i]
                    label_features[label] = torch.cat((label_features[label], features[i][None]))

            # Create weight for the last layer
            #print(label_features[0].size())
            for j in range(n_way):
                feature_avg = torch.mean(label_features[j], 0)
                #print(feature_avg.size())
                fc_weight = torch.cat((fc_weight, feature_avg[None]), 0)

            # Apply weight to the model
            nc_dict = model.state_dict()
            #print(nc_dict['cos_sim.weight'])
            nc_dict['cos_sim.weight'] = fc_weight
            #print(nc_dict['cos_sim.weight'])
            #print(nc_dict['cos_sim.weight'].size())
            #print(fc_weight)
            model.load_state_dict(nc_dict)

        # Testing
        val_loss, acc, m_acc, macro_f1, class_f1 = test(model, test_loader, criterion, device, n_way)

        if (save_models):
            torch.save(model.state_dict(), os.path.join(path_models, f'cosine_{epoch + 1}.pth'))  # Save the model

        # Append and report results
        #df_results.loc[epoch] = [val_loss, acc, m_acc, macro_f1] + class_f1
        print(f'epoch: {stuff} v_loss: {val_loss:.5f} val_acc: {acc:.5f} val_m_acc: {m_acc:.5f} f1: {macro_f1:.5f}')

        #df_results.to_csv(path_results, index=False)  # Export results to a CSV file

if __name__ == '__main__':
    main()


epoch: 1 v_loss: 1.10031 val_acc: 0.33889 val_m_acc: 0.33889 f1: 0.31230
epoch: 2 v_loss: 1.09927 val_acc: 0.34444 val_m_acc: 0.34444 f1: 0.31651
epoch: 3 v_loss: 1.09884 val_acc: 0.35111 val_m_acc: 0.35111 f1: 0.32365
epoch: 4 v_loss: 1.09913 val_acc: 0.36000 val_m_acc: 0.36000 f1: 0.33292
epoch: 5 v_loss: 1.09817 val_acc: 0.36222 val_m_acc: 0.36222 f1: 0.33639
epoch: 6 v_loss: 1.09791 val_acc: 0.35889 val_m_acc: 0.35889 f1: 0.33493
epoch: 7 v_loss: 1.09948 val_acc: 0.35667 val_m_acc: 0.35667 f1: 0.33444
epoch: 8 v_loss: 1.09931 val_acc: 0.35667 val_m_acc: 0.35667 f1: 0.33445
epoch: 9 v_loss: 1.09820 val_acc: 0.35889 val_m_acc: 0.35889 f1: 0.33734
epoch: 10 v_loss: 1.09833 val_acc: 0.35889 val_m_acc: 0.35889 f1: 0.33922
epoch: 11 v_loss: 1.09821 val_acc: 0.35667 val_m_acc: 0.35667 f1: 0.33758
epoch: 12 v_loss: 1.09838 val_acc: 0.35889 val_m_acc: 0.35889 f1: 0.34024
epoch: 13 v_loss: 1.09821 val_acc: 0.36000 val_m_acc: 0.36000 f1: 0.34120
epoch: 14 v_loss: 1.09800 val_acc: 0.35667 val_

In [7]:
from basic_train import *


def main():
    # Set Training Parameters
    num_workers = 12
    bs = 64
    n_way = 3
    path_pretrained = '../results/basic_cosine/basic_cosine_55.pth'
    save_models = True  # Whether to save the trained models (Occurs every epoch)
    k_shot = 20  # Must have the generated split to match it

    path_splits = f'../splits/{k_shot}_shot.csv'  # Location of preprocessed splits
    path_results = f'../../results/{k_shot}shot_nc_cs.csv'  # Full path to save the CSV results
    path_models = f'../../models/nc_cs/{k_shot}_shot'  # Folder path to save the trained models to

    torch.cuda.set_device(0)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    criterion = nn.CrossEntropyLoss()
    
    # Load in model
    model = CosineSimilarityNet(n_way).to(device)
    pretrained_dict = torch.load(path_pretrained)
    #del pretrained_dict['linear.weight']  # Remove the last layer
    #del pretrained_dict['linear.bias']
    del pretrained_dict['cos_sim.weight']
    model_dict = model.state_dict()
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

    # Load in data
    train_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                                csv_path=path_splits, mode='novel_train', resize=224)
    test_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                               csv_path=path_splits, mode='novel_validate', resize=224)
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)

    # Create Dataframe to export results to CSV
    df_results = pd.DataFrame(columns=['Validation Loss', 'Accuracy', 'Macro Accuracy',
                                       'Macro-F1 Score'] + [str(x) + ' F1' for x in range(n_way)])
    
    # Find Average Features
    label_features = [torch.FloatTensor([]).to(device) for i in range(n_way)]
    model.eval()
    fc_weight = torch.FloatTensor([]).to(device) # Initialise weight for the nearest centroid, last layer weight
    with torch.no_grad():
        for step, (data_inputs, data_labels) in enumerate(train_loader):
            # Get Features
            inputs, labels = data_inputs.to(device), data_labels.to(device)
            _, features = model(inputs, extract_features=True)
            
            # Sort features into labels
            for i in range(features.size(0)):
                label = labels[i]
                label_features[label] = torch.cat((label_features[label], features[i][None]))
        
        # Create weight for the last layer
        for j in range(n_way):
            feature_avg = torch.mean(label_features[j], 0)
            fc_weight = torch.cat((fc_weight, feature_avg[None]), 0)
            
        # Apply weight to the model
        nc_dict = model.state_dict()
        nc_dict['cos_sim.weight'] = fc_weight
        model.load_state_dict(nc_dict)
        
    # Testing
    val_loss, acc, m_acc, macro_f1, class_f1 = test(model, test_loader, criterion, device, n_way)

    if (save_models):
        torch.save(model.state_dict(), os.path.join(path_models, f'nc_cs.pth'))  # Save the model

    # Append and report results
    df_results.loc[0] = [val_loss, acc, m_acc, macro_f1] + class_f1
    print(f'v_loss: {val_loss:.5f} val_acc: {acc:.5f} val_m_acc: {m_acc:.5f} f1: {macro_f1:.5f}')

    df_results.to_csv(path_results, index=False)  # Export results to a CSV file

if __name__ == '__main__':
    main()


v_loss: 1.08115 val_acc: 0.45556 val_m_acc: 0.45556 f1: 0.42568


In [16]:
from basic_train import *

# Set Training Parameters
num_epochs = 100
num_workers = 12
bs = 64
n_way = 3
path_pretrained = '../results/basic_cosine/basic_cosine_58.pth'
save_models = False  # Whether to save the trained models (Occurs every epoch)
freeze = ['cos_sim.weight']  # Freeze all layers except linear layers
k_shot = 20  # Must have the generated split to match it

path_splits = f'../splits/{k_shot}_shot.csv'  # Location of preprocessed splits
path_results = f'../../results/{k_shot}shot_cosine.csv'  # Full path to save the CSV results
path_models = f'../../models/cosine_similarity/{k_shot}_shot'  # Folder path to save the trained models to

torch.cuda.set_device(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load in model
model = BaselineNet(n_way).to(device)
pretrained_dict = torch.load(path_pretrained)
del pretrained_dict['cos_sim.weight']  # Remove the last layer
model_dict = model.state_dict()
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [25]:
for name, param in model.namedparameters():
    print(param.size())

torch.Size([64, 1, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([64, 64, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([3, 12544])
torch.Size([3])


In [5]:
for i in range(9,17):
    print(i)

9
10
11
12
13
14
15


In [10]:
1e-4 == 0.0001

True

In [ ]:
    # Testing
    for epoch in range(num_epochs):
        # train_loss = train(model, train_loader, criterion, device, optimizer, freeze=freeze)
        val_loss, acc, m_acc, macro_f1, class_f1 = test(model, test_loader, criterion, device, n_way)

        if (save_models):
            torch.save(model.state_dict(), os.path.join(path_models, f'cosine_{epoch + 1}.pth'))  # Save the model

        # Append and report results
        df_results.loc[epoch] = [epoch + 1, train_loss, val_loss, acc, m_acc, macro_f1] + class_f1
        print(
            f'[{epoch + 1}] t_loss: {train_loss:.5f} v_loss: {val_loss:.5f} val_acc: {acc:.5f} '
            f'val_m_acc: {m_acc:.5f} f1: {macro_f1:.5f}')

    #df_results.to_csv(path_results, index=False)  # Export results to a CSV file